In [121]:
import requests
import pandas as pd
import plotly.graph_objects as go

url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"

filtros ={
    "vs_currency": "usd",
    "days": "30",
    "interval": "daily"
}

In [122]:
resposta = requests.get(url, params=filtros)
dados = resposta.json()

df = pd.DataFrame(dados["prices"], columns=["Timestamp", "Preco"])
df["Data"] = pd.to_datetime(df["Timestamp"], unit = "ms")

valor_inicial = df["Preco"].iloc[0]
valor_final = df["Preco"].iloc[-1]
variacao = ((valor_final - valor_inicial) / valor_inicial) * 100

valor_max = df["Preco"].max()
data_max = df.loc[df["Preco"].idxmax(), "Data"]

valor_min = df["Preco"].min()
data_min = df.loc[df["Preco"].idxmin(), "Data"]

valor_medio = df["Preco"].mean()

In [123]:
import sqlite3

conexao = sqlite3.connect('bitcoin_dados.db')
df.to_sql('precos_btc', conexao, if_exists='replace', index=False)

query = "SELECT * FROM precos_btc"
df_sql = pd.read_sql(query, conexao)

In [124]:
valor_inicial = df_sql["Preco"].iloc[0]
valor_final = df_sql["Preco"].iloc[-1]
variacao = ((valor_final - valor_inicial) / valor_inicial) * 100

valor_max = df_sql["Preco"].max()
data_max = df_sql.loc[df_sql["Preco"].idxmax(), "Data"]

valor_min = df_sql["Preco"].min()
data_min = df_sql.loc[df_sql["Preco"].idxmin(), "Data"]

valor_medio = df_sql["Preco"].mean()

In [125]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = df_sql["Data"],
    y = df_sql["Preco"],
    mode = "lines+markers",
    name = "Preço do BTC",
    line = dict(color = "orange")
))


fig.update_layout(
    title = "Análise de Preço do Bitcoin - Últimos 30 dias",
    xaxis_title = "Data",
    yaxis_title = "Preço (USD)",
    template = "plotly_dark"
)

fig.add_annotation(
    x = data_max,
    y = valor_max,
    text = f"Máxima: ${valor_max:,.2f}",
    showarrow = True,
    arrowhead = 1
)

fig.add_annotation(
    x = data_min,
    y = valor_min,
    text = f"Mínima: ${valor_min:,.2f}",
    showarrow = True,
    arrowhead = 1
)

fig.show()

## Resumo do Período Analisado

Nesta seção, apresento os principais indicadores de preço do Bitcoin, recuperados diretamente do banco de dados SQL.

In [126]:
# @title
print("-" * 35)
print(f"💵 Preço Atual: ${valor_final:,.2f}")
print(f"📊 Variação no Período: {variacao:.2f}%")
print(f"💸 Preço Médio: US$ {valor_medio:,.2f}")
print(f"📈 Preço Máximo: US$ {valor_max:,.2f}")
print(f"📉 Preço Mínimo: US$ {valor_min:,.2f}")
print("-" * 35)

-----------------------------------
💵 Preço Atual: $69,301.23
📊 Variação no Período: -24.06%
💸 Preço Médio: US$ 86,869.26
📈 Preço Máximo: US$ 97,007.78
📉 Preço Mínimo: US$ 62,853.69
-----------------------------------


##  Detalhamento Técnico (SQL)
Utilizando **Subqueries em SQL**, filtrei abaixo os dias em que o preço fechou inferior à média do período.

In [127]:
# @title
query_insights = "SELECT Data, Preco FROM precos_btc WHERE Preco < (SELECT AVG(Preco) FROM precos_btc)"
df_insights = pd.read_sql(query_insights, conexao)

df_insights["Data"] = pd.to_datetime(df_insights["Data"]).dt.strftime("%d/%m/%Y")
df_insights["Preco"] = df_insights["Preco"].round(2)

display(df_insights)
conexao.close()

,Data,Preco
0,26/01/2026,86548.32
1,30/01/2026,84570.41
2,31/01/2026,84141.78
3,01/02/2026,78725.86
4,02/02/2026,76937.06
5,03/02/2026,78767.66
6,04/02/2026,75638.96
7,05/02/2026,73172.29
8,06/02/2026,62853.69
9,06/02/2026,69301.23
